<a href="https://colab.research.google.com/github/japan-medical-ai/medical-ai-course-materials/blob/master/source/source/notebooks/Basenji.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![colab-logo](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/japan-medical-ai/medical-ai-course-materials/blob/master/notebooks/Basenji.ipynb)

# 実践編：ディープラーニングを使った配列解析

近年，次世代シーケンサ（NGS; Next Generation Sequencer）の発展により，遺伝子の塩基配列を高速，大量，安価に読み取ることができるようになってきました．

ここではディープラーニングを用いて，DNA配列からエピジェネティックな影響や転写制御を予測する問題に取り組みます．この予測モデルを使うことで，ある遺伝子変異が遺伝子発現にどのような影響を与えるのかを予測することができるようになります．










## 環境

ここで用いるライブラリは


*  Chainer
*  Cupy
*  matplotlib

です．Google Colab上では，以下のようにしてインストールすることができます．以下のセルを実行（Shit+Enter）してください．


In [0]:
!set -ex
!apt -y -q install cuda-libraries-dev-9-2
!pip install cupy-cuda92 --pre
!pip install chainer --pre

インストールが完了したら，以下のセルを実行して，各ライブラリのバージョンを確認してください．



In [0]:
import chainer
import cupy
import matplotlib

chainer.print_runtime_info()
print('matplotlib:', matplotlib.__version__)

```
Platform: Linux-4.14.65+-x86_64-with-Ubuntu-18.04-bionic
Chainer: 6.0.0a1
NumPy: 1.14.6
CuPy:
  CuPy Version          : 6.0.0a1
  CUDA Root             : /usr/local/cuda
  CUDA Build Version    : 9020
  CUDA Driver Version   : 9020
  CUDA Runtime Version  : 9020
  cuDNN Build Version   : 7201
  cuDNN Version         : 7201
  NCCL Build Version    : 2213
iDeep: Not Available
('matplotlib:', '2.1.2')
```

## 配列解析について

次世代シーケンサの発展・普及とともに，大量の遺伝子配列が読み取られるようになりました．そうした中で，塩基配列で表現された遺伝子型と病気や形態などの表現型との関係を推定するようなGWAS（Genome Wide Association Study; ゲノムワイド関連解析）がされてきましたが，遺伝子変異だけでは全ての表現型の変化を説明できないことがわかってきています．特に，非翻訳領域が遺伝子発現に影響を与え，表現型の変化を生じさせていることが様々な実験結果からわかってきています．遺伝子発現時に周辺領域がどのように影響を与えているのかを調べるために様々な手法が提案されています．（以下図）

![代替テキスト](https://www.encodeproject.org/images/c45f4d8c-0340-4fcb-abe3-e4ff0bb919be/@@download/attachment/EncodeDatatypes2013-7.png)[Encode Projectより引用]

例えば，ChIP-Seq（クロマチン免疫沈降）は，転写調節因子やそのほかのタンパク質が直接の相互作用を起こすDNAの特定部位を分離し，それらをシーケンシングして同定し，どの程度出現していたかを定量化します．これにより，タンパク質のDNA中の結合部位を正確かつ効率的に同定することができます．

このような技術で抽出された配列を学習データとして利用し，DNA配列のみからそこが結合部位かどうかだけでなく，どの程度，出現していたのかというカバレッチ値を推定することで，DNA配列のみから，転写因子，クロマチンアクセシビリティ，ヒストン修飾を予測することができるようになり，様々な遺伝子変異に対する有益な洞察を与えてくれます．

一方で，DNA配列中のどの領域がそのような特徴を持つのかを調べるためには非常に遠距離のDNA配列もいる必要があり，これが機械学習による解析を困難としていました．今回紹介する手法はこのような遠距離の関係を捉えるため，10万超の長さのDNA配列を入力として受け取り，128bpごとにその領域がどの程度各手法で発現していたのか，カバレッジ値を予測するタスクを考えます．

## データセット

ここでは，データセットとして[FANTOM5](http://fantom.gsc.riken.jp/5/)のCAGEデータセットを利用します．ここでは前処理が既に終わって配列と，各位置ごとの推定カバレッジ値が記録されたデータを利用します．下のセルを実行してデータをダウンロードしてください．（TODO: 公開の場所においてcurlで取得するようにする）



In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


file_id = "1lQk10NsD_NRELz5L0v76TngTPZowHlMB"

drive_file = drive.CreateFile({'id': file_id})


drive_file.GetContentFile("data.h5")




```
total 3640080
-rw-r--r-- 1 root root       2597 Oct 17 05:38 adc.json
-rw-r--r-- 1 root root 3727429632 Oct 17 05:39 data.h5
drwxr-xr-x 2 root root       4096 Oct 15 20:47 sample_data
```


In [0]:
!ls -lh

data.h5というファイルが正しくダウンロードされているかを確認してください．

data.h5はHDF5形式でデータを格納したファイルです．HDF5ファイルは，ファイルシステムと同様に，階層的にデータを格納することができ，行列やテンソルデータをそれぞれの位置で名前付きで格納することができます．

HDF5形式のファイルを操作するためにh5pyというライブラリがあります．h5pyのFile()でファイルを開き，keys()というAPIでその中に含まれているキーを列挙します．取得したキーを使って格納されている各データを参照することができます．

テンソルデータはnumpyと同様にshapeという属性でそのサイズを取得することができます．

以下のセルを実行して格納されているデータを確認してください．

In [0]:
import h5py

with h5py.File('data.h5', 'r') as hf:
  for key in hf.keys():
    print(key, hf[key].shape)



```
(u'pool_width', ())
(u'target_ids', (3,))
(u'target_labels', (3,))
(u'target_strands', (3,))
(u'test_in', (444, 131072, 4))
(u'test_out', (444, 1024, 3))
(u'test_out_full', (444, 1024, 3))
(u'train_in', (6240, 131072, 4))
(u'train_out', (6240, 1024, 3))
(u'valid_in', (338, 131072, 4))
(u'valid_out', (338, 1024, 3))
```



In [0]:
!ls -l

h5py形式のファイルをnumpyデータとして扱うには，コピーする必要があります．以下のコードは'train_in'というキーに対応するテンソルデータをnumpyデータとして読み出し，そのデータを一部を表示します．

In [0]:
with h5py.File('data.h5') as hf:
  x = hf['train_in'][:100]
  print(x.shape, x.dtype)
  print(x[0, 0:10])
  y = hf['train_out'][:100]
  print(y.shape)
  print(y.shape, y.dtype)
  print(y[0, 0:10])

## Dilated Convolutionを用いた解析

### 配列解析の戦略

配列データを扱うためには大きく３つの戦略があります．

一つ目は，配列中の順序情報は捨てて，配列をその特徴の集合とみなすことです．これはBag of Words（BoW）表現とよびます．このBoW表現は特徴に十分情報が含まれていれば強力な手法ですがDNA配列のような4種類の文字からなる配列やその部分配列だけではその特徴を捉えることは困難です．

二つ目は配列中の要素を左から右に順に読み込んでいき計算していく手法です．これはRNNを用いて解析します．このRNNの問題点はその計算が逐次的であり計算量が配列長に比例するという点です．現在の計算機は計算を並列化することで高速化を達成していますがRNNは計算を並列化することが困難です．もう一つの問題は遠距離間の関係を捉えることが難しいという点です．RNNはその計算方式から，計算の途中結果を全て状態ベクトルに格納する必要があります．遠距離間の関係を捉えようとすると，多くの情報を覚えておかなければなりませんが状態ベクトルサイズは有限なので，多くの情報を忘れなければなりません．

三つ目は配列データを1次元の画像とみなし，画像処理の時と同様にCNNを用いて解析する手法です．CNNはRNNの場合と違って各位置の処理を独立に実行できるため並列に処理することができます．また，後述するDilated Convolutionを使うことで各位置の処理は遠距離にある情報を直接読み取ることができます．次の章でDilated Convolutionについてさらに詳しくみていきます．






### Dilated Convolution

従来の畳み込み層を使って配列解析をする場合を考えてみます．
以下の図のようにある位置の入力の情報は各層で隣接する位置からしか読み込まれません．どのくらい離れた位置から情報を取得するかはカーネルサイズによって決定され，カーネルサイズがKの時，Dだけ離れた距離にある情報を取得するためにはD/K層必要となります．今回の問題の場合Dは数百から数万，Kは3や5といった値ですので必要な層数も百から万といった数になってしまい現実的ではありません．

![orig conv](http://musyoku.github.io/images/post/2016-09-17/naive_conv.png)

[WaveNet: A Generative Model for Raw Audio](https://deepmind.com/blog/wavenet-generative-model-raw-audio/)より引用

それに対し，Dilated Convolution（atrous convolutionやconvolution weith holesともよばれます）は読み取る場所をずらしたところからうけとります．例えばDilation=4の場合，4だけ離れた位置から情報を受け取ります．このDilatedを倍々にしていき，カーネルサイズを2とした場合，Dだけ離れた位置の情報を受取るには log_2 D層だけ必要になります．今回のDが数百から数万の場合，10から20層程度あれば済むことになります．

今回はこのDilated Convolutionを使うことで遠距離にある情報を考慮できるモデルを作成します．

![dilated convolution](https://storage.googleapis.com/deepmind-live-cms/documents/BlogPost-Fig2-Anim-160908-r01.gif)

[Wavenet Blog記事](https://deepmind.com/blog/wavenet-generative-model-raw-audio/)より引用


### ブロック

それでは最初に，ネットワークの全体を設計します．
このネットワークは二つのブロックから構成されます．

１つ目のブロックは長さが2^17から配列を長さが2^10のベクトル列まで圧縮し，128bpにつき1つのベクトルが対応するにします．この圧縮はRootBlockが担当します．

二つ目のブロックは遠距離にある情報を考慮して各ベクトルの値を計算する部分を担当します．

以下のコードを実行してみましょう．


In [0]:
import chainer
import chainer.functions as F
import chainer.links as L
import cupy as cp


default_squeeze_params = [
  # out_ch, kernel, pool
  [32, 21, 2], #1 128 -> 64
  [32, 7, 4], #2 64 -> 16
  [48, 7, 4], #3 16 -> 4
  [64, 7, 4]  #4 4 -> 1
]


default_dilated_params = [
# out_ch, kernel, dilated
  [8, 3, 1],
  [8, 3, 2], 
  [8, 3, 4], 
  [8, 3, 8], 
  [8, 3, 16], 
  [8, 3, 32],
  [8, 3, 64]
]

class Net(chainer.Chain):
    
  def __init__(self, squeeze_params=default_squeeze_params, dilated_params=default_dilated_params, n_targets=3):
    super(Net, self).__init__()
    self._n_squeeze = len(squeeze_params)
    self._n_dilated = len(dilated_params)
    with self.init_scope():
      for i, param in enumerate(squeeze_params):
        out_ch, kernel, pool = param
        setattr(self, "s_{}".format(i), SqueezeBlock(out_ch, kernel, pool))
      for i, param in enumerate(dilated_params):
        out_ch, kernel, dilated = param
        setattr(self, "d_{}".format(i), DilatedBlock(out_ch, kernel, dilated))
      self.l = L.ConvolutionND(1, None, n_targets, 1)
    
  def forward(self, x):
    # x : (B, X, 4)    
    xp = cp.get_array_module(x)
    h = xp.transpose(x, (0, 2, 1))
    h = h[:, :, :]
    h = h.astype(xp.float32)
                
    for i in range(self._n_squeeze):
      h = self["s_{}".format(i)](h)
    
    hs = [h]
    for i in range(self._n_dilated):
      h = self["d_{}".format(i)](hs)
      hs.append(h)

    h = self.l(F.concat(hs))
    h = xp.transpose(h, (0, 2, 1))
    return h

このネットワークは初期化時の引数としてRootBlockに関するパラメータと，DilatedBlockに関するパラメータを受け取ります．

それぞれ，出力チャンネル，カーネルサイズ，プーリングの三つ組からなるリストと，出力チャンネル，カーネルサイズ，dilatedサイズの三つ組からなるリストを受け取ります．

次に，ブロックの定義をします．

In [0]:
import chainer
import chainer.functions as F
import chainer.links as L
import cupy as cp

class SqueezeBlock(chainer.Chain):  
  def __init__(self, out_ch, kernel, pool):
    super(SqueezeBlock, self).__init__()
    
    self.pool = pool
    with self.init_scope():
        pad = kernel // 2
        self.conv = L.ConvolutionND(1, None, out_ch, kernel, pad=pad, nobias=True)
        self.bn = L.BatchNormalization(out_ch)
      
  def forward(self, x):
    h = self.conv(x)
    h = self.bn(h)
    h = F.max_pooling_nd(h, self.pool, self.pool, 0, cover_all=False)
    return F.relu(h)

class DilatedBlock(chainer.Chain):
  def __init__(self, out_ch, kernel, dilate):
    super(DilatedBlock, self).__init__()
    with self.init_scope():
      self.conv = L.ConvolutionND(1, None, out_ch, kernel, pad=dilate, nobias=True, dilate=dilate)
      self.bn = L.BatchNormalization(out_ch)
      
  def forward(self, xs):
    h = self.conv(F.concat(xs))
    h = self.bn(h)    
    return F.relu(h)

それでは，試しにネットワークを構築して，そこにサンプルデータを流してみましょう．



In [0]:
import numpy as np
n = Net()
size = 131072 # 128 * 1024
batchsize = 4
x = np.empty((batchsize, size, 4), dtype=np.bool)
y = n.forward(x)
print(y.shape)

(4, 1024, 3)


```
(4, 1024, 3)
```



ここで，もともとB= 4, L=131072, C=4だった配列が計算後はB=4, L=1024, C=3の配列となりました．

今回の学習では対数ポアソン損失関数を利用します．これはモデルはポアソン分布の唯一のパラメータである平均を出力し，そのポアソン分布を学習データを使った最尤推定をします．この際，学習対象パラメータ以外は無視しています．性能評価する際には，比較がしやすいように学習パラメータに依存しない項も含めた式を利用しています．

In [0]:
import chainer.functions as F
import math
import sklearn

def log_poisson_loss(log_x, t):
  return F.mean(F.exp(log_x) - t * log_x)
  

def log_poisson_acc(log_x, t):
  xp = cp.get_array_module(t)
  t = t.astype(xp.float32)
  zeros = xp.zeros_like(t, dtype=t.dtype)
  ones = xp.ones_like(t, dtype=t.dtype)
  cond = xp.logical_and(t >= zeros, t <= ones)
  stirling_approx = t * F.log(t) - t + 0.5 * F.log(2.0 * math.pi * t)
  ret = F.exp(log_x) - t * log_x + F.where(cond, zeros, stirling_approx)
  return F.mean(ret)

これで全部準備ができました．残りはchainerのtrainerを改造して学習するだけです．以下のコードを実行してください．30分程度で学習が完了します．

In [0]:
import chainer
import chainer.functions as F
import chainer.links as L
import numpy as np
from chainer.training import extensions
from chainer import training
import h5py

ml_h5 = h5py.File('data.h5')
print(list(ml_h5.keys()))

train_x = ml_h5['train_in']
train_y = ml_h5['train_out']

valid_x = ml_h5['valid_in']
valid_y = ml_h5['valid_out']

test_x = ml_h5['test_in']
test_y = ml_h5['test_out']

train = chainer.datasets.TupleDataset(train_x, train_y)
val = chainer.datasets.TupleDataset(valid_x, valid_y)

train = train[:len(train)//10]
val = val[:len(val)//10]

batchsize = 8

train_iter = chainer.iterators.SerialIterator(train, batchsize)
val_iter = chainer.iterators.SerialIterator(val, batchsize, repeat=False, shuffle=False)

model = L.Classifier(Net(), lossfun=log_poisson_loss, accfun=log_poisson_acc)


optimizer = chainer.optimizers.Adam(alpha=0.002, beta1=0.97, beta2=0.98)
optimizer.setup(model)


updater = training.updaters.StandardUpdater(
  train_iter, optimizer, device=0)

epoch = 4
out = "out"
trainer = training.Trainer(updater, (epoch, 'epoch'), out=out)

trainer.extend(extensions.Evaluator(val_iter, model, device = 0))
trainer.extend(extensions.LogReport())
trainer.extend(extensions.snapshot_object(model, 'model_iter_{.updater.iteration}'), trigger=(100, 'iteration'))

trainer.extend(extensions.PrintReport(
    ['epoch', 'main/loss', 'validation/main/loss', 'main/true_loss', 'validation/main/true_loss', 'elapsed_time']), trigger = (0.1, 'epoch'))

trainer.extend(extensions.ProgressBar())
     
trainer.run()


In [0]:
import chainer
import chainer.links as L
%matplotlib inline
import matplotlib.pyplot as plt

model_iter = 300
out_dir = 'out'
model = L.Classifier(Net(), lossfun=log_poisson_loss, accfun=log_poisson_acc)
chainer.serializers.load_npz('{}/model_iter_{}'.format(out_dir, model_iter), model)
predictor = model.predictor

print(len(test_x))
with chainer.no_backprop_mode():
  test_y_estimated = predictor(test_x[:1])
print(test_y_estimated.shape)  





In [0]:
def plot(y):
  print(y.shape)
  ret = np.reshape(np.transpose(y, (2, 1, 0)), (3, -1))
  for i in range(3):
    print(ret[i].shape, type(ret[i]))
    plt.plot(range(len(ret[i])), ret[i])
    
plot(test_y_estimated)